# Building a regression model for predicting house sale prices

In [ ]:
import pickle
import pathlib
import numpy as np
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

In [ ]:
clean_data_path = DATA_DIR / 'processed' / 'ames_transformed.pkl'

In [ ]:
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [ ]:
data.info()

In [ ]:
model_data = data.copy()

## Preparing the data for the model

Now that we have the data all cleaned, and all the missing values accounted for, lets focus on transforming the data for the model.

Lets remember what a model is. 

- A predictive model is a **set** of functions that receive data as input and produce a prediction, that is, an estimate of the target value as output.
- **Train** a model is to search the set of candidate functions for one that adequately represents the **training dataset**.
- The adequacy of a candidate function to the training data is usually determined by a **loss function** that measures how well the predictions of the function match the real values of the target within the training dataset. It is common to define a *loss function per data item* (e.g. absolute error, quadratic error, etc) and to construct the *loss function over the dataset* as the *average prediction loss*.

Many models are **parametric models**. In this case, each function of the set of functions that makes the model is constructed from a vector of **parameters** that define the function, forming a **parametric function**. For instance: the linear model constructs prediction values out of a linear combination of the input features, plus a constant. The weights of the linear combination plus the constant are the parameters of the model. The set of functions that can be represented by this model is given by all possible values of the vector of parameters that define the function.

Some models are called **non-parametric models**. These models usually do not have a parametric form (like the linear model). But the terminology is a bit misleading, though: usually these models *do* have parameters, and potentially an open-ended set of them! For instance, consider the "decision tree" model, which is one of the most prominent models of this category. The decision tree may not have a formula for the predicted value, but it does have parameters, many of them: each decision in the tree involves a choice of feature and a threshold level, and those choices must be stored as parameters of the model for use in future predictions.

Each model has specific requirements for the format of the input data. Most of the time, the minimum requirement is that:

- All columns are numeric;
- There are no missing values.

Some models have extra requirements. For example: the support-vector-machines model requires that the input features have comparable standard deviations - having features that have large discrepancies between features in terms of their order of magnitude (such as a feature in the fractions of unit range and another in the tens of thousands) will result in poor prediction quality.

And some models may not have any special requirement at all. We will study each of those in detail in this course.

Lets start our study with a simple model: the *multivariate linear regression* model. This is a model that presents the minimum requirements listed above. So we need to do a bit of processing on the original features:

- *Numerical features* stay as given;
- *Categorical features* have to be transformed into numerical features. In order to do so we need to **encode** these features, that is: to transform them into new features that convey the same information, but in a numerical form, and in a way that "makes sense" - we'll see it below.
- *Ordinal features* can be transformed into numerical features in the same way as the caegorical features, or could be assigned increasing numbers in conformity with the ordered nature of the categories of the feature.

## Encoding categorical variables

Lets identify all categorical variables - both nominal (that is, categoricals without category order) and ordinal.

In [ ]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [ ]:
ordinal_columns

In [ ]:
categorical_columns

### Encoding ordinal variables 

Ordinal variables can be transformed into integer numbers in a straightforward manner: the lowest category is assigned the value "zero", the next category is given the value "one", etc. The `Pandas` library has a function for this task: `factorize()`:

In [ ]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes / codes.max() ## Acling on the data

Lets confirm that the variables are no longer ordinal, but now are integers:

In [ ]:
model_data[ordinal_columns].info()

Compare the original values with the encoded values:

In [ ]:
data['Lot.Shape'].value_counts()

In [ ]:
model_data['Lot.Shape'].value_counts()

In [ ]:
model_data[ordinal_columns].head(10)

### Encoding nominal variables

With nominal variables there is no notion of order among categories. Therefore, it would be a conceptual mistake to encode them in the same manner as the ordinal variables. For instance, consider the `Exterior` variable:

In [ ]:
model_data['Exterior'].value_counts()

We cannot assign an order here, lest we end up with equations like `HdBoard` + `Plywood` = `CemntBd`, which are nonsense. 

The strategy here to encode `Exterior` is to create several new numerical variables to represent the membership of a given data item to one of the `Exterior` categories. These are called **dummy variables**. Each of these new variables contain only the values "zero" or "one" (i.e. they are binary variables), where $1$ denotes that the data item belongs to the category represented by the variable. Evidently, for a given data item, only one dummy variable has a value of $1$, all remaining are $0$.

There are two types of dummy variable encoding:

- "One-hot" encoding: in this case we create one dummy variable per category. Let's look at the `Exterior` feature as an example. The `Pandas` function `get_dummies()` can do the encoding for us:

In [ ]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

Observe that for each value of `Exterior`, only the corresponding dummy is flagged.

One-hot encoding is a popular technique in Machine Learning. Statisticians, however, prefer a slightly different way of dummy encoding which is:

- Choose a category to *not encode* (this is called the *base category*)
- Generate dummies for the remaining categories. That is:
    - If the data item belongs to the base category, no dummy receives a value of $1$;
    - Otherwise, set the corresponding dummy to $1$.

The same `get_dummies()` function of `Pandas` can do this automatically with the `drop_first` argument:

In [ ]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

Notice that we are now missing the dummy variable for the `AsbShng` category.

Why to encode things this way? If we don't drop one of the dummies, then it will always be the case that the sum of the values of the dummies is $1$ (since each data item must belong to one of the categories). The linear model, particularly very popular with the statisticians, implies the existence of a fictitious feature containing, for all data items, the value $1$. Hence we end up having a set of variables where a linear combination of them (in this case, the sum of the dummies) matches the value at another variable. This has numerical computing implications for the linear model, that we will discuss in class.

Since we want to use the linear model in this notebook, lets encode all categoricals with the `drop_first` alternative.

In [ ]:
model_data = pd.get_dummies(model_data, drop_first=True)

Now our dataset has a lot more variables!

In [ ]:
model_data.info()

In [ ]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

## Train-test splitting

The data will now be organized as follows:

- The features form a matrix $X$ of size $m \times n$, where $m$ is the number of data items, and $n$ is the number of features.
- The target forms a column-matrix $y$ of length $m$.

In [ ]:
X = model_data.drop(columns=['SalePrice']).copy().values
y = model_data['SalePrice'].copy().values

In [ ]:
X, y

This is the typical set-up of a machine learning project. Now we want to train our model *and* verify that the model provides good predictions for *unseen* data items. Why the emphasis on "unseen"? Because there is no use for a model that only gives predictions for the items in the data used to train it - we want our models to *generalize*.

The way to assess the model's performance for unseen values is to split the dataset into two subsets: the **training** and **test** datasets.

We have been using a lot of `Pandas` to manipulate our data so far. From now on we will switch to another very popular library for machine learning in Python: `Scikit-Learn`.

The function `train_test_split()` will take as arguments the dataset to be split, the specification of the fraction of the dataset to be reserved for testing, and a random seed value - so that the split will always be the same whenever we run our notebook. This is a customary measure to ensure reproducibility of the notebook.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42  # Any number here, really.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [ ]:
X.shape, Xtrain.shape, Xtest.shape

In [ ]:
y.shape, ytrain.shape, ytest.shape

## Fitting some models

There are lots of different models to choose from. In this notebook we are going to fit a few of them and compare their performance. We will start with our baseline model, the linear regressor, then try Decision Tree, Random Forest and Elastic Net. We will also try to improve those models by tuning their hyperparameters.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class DummyModel(BaseEstimator, ClassifierMixin):

    def init(self):
        self.mean = 0

    def fit(self, X, y):
        self.mean = y.mean()
        return self

    def predict(self, X):
        return self.mean[0] * np.ones(shape=[X.shape[0], 1])

In [ ]:
BaselineModel = LinearRegression()
#DummyModel = DummyModel()
RandomForestModel = RandomForestRegressor()
DecisionTreeModel = DecisionTreeRegressor()
ElasticNetModel = ElasticNet()

results = {}

In [ ]:
from sklearn.model_selection import cross_val_score

# Train and evaluate baseline model
baselineModel.fit(Xtrain, ytrain)
baselineScore = -cross_val_score(baselineModel, Xtrain, ytrain, cv=5, scoring='neg_mean_squared_error')

results['Baseline'] = {
    'best_params': None,
    'best_score': baselineScore.mean(),
    'best_model' : baselineModel
}

results['Baseline']

In [ ]:
params_RandomForest = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [None, 5, 10], 
    'min_samples_split': [1, 2, 5], 
    'min_samples_leaf': [1, 2, 4], 
    'bootstrap': [True, False],
    'random_state': [42]
}

params_ElasticNet = {
    'alpha': np.arange(0, 0.01, 0.001),  
    'l1_ratio': np.arange(0, 0.01, 0.001)  
}

params_DecisionTree = {
    'max_depth': np.arange(8, 12, 1),  
    'min_samples_split': np.arange(0, 5, 1),  
    'min_samples_leaf': np.arange(8, 12, 1), 
    'random_state': [42]
}

paramsList = [params_ElasticNet,
              params_DecisionTree,
              params_RandomForest]

modelsList = [ElasticNetModel,
              DecisionTreeModel,
              RandomForestModel]

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
import time

for model, params in zip(modelsList,paramsList):
    
    # grid search for each model and parameters
   
    grid_search = GridSearchCV(
        model,
        params,
        cv=5,
        scoring='neg_mean_squared_error',
        return_train_score=True,
        n_jobs=-1,
    )

    t1 = time.perf_counter()
    grid_search.fit(Xtrain, ytrain)
    t2 = time.perf_counter()

    # Store the results
    results[type(model).__name__] = {
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_model' : grid_search.best_estimator_
    }
    
    print(results[type(model).__name__]['best_params'])
    print(f"Finished {type(model).__name__} in {t2-t1:.2f} seconds.")

## Displaying model Results

In [ ]:
# Sorting the results based on mean test score
sorted_results = sorted(results.items(), key=lambda x: -x[1]['best_score'])

params = {}

for model_name, result in sorted_results:
    params[model_name] = result['best_params']

    if model_name != 'Baseline':

        print(f"Results for {model_name}:")
        print(f"Best Parameters: {result['best_params']}")
        print(f"Best Score RMSE: {np.sqrt(-result['best_score'])}")
        print(f"Mean Test Score %: {100*((10**(np.sqrt(-result['best_score'])))-1)}")
        print("=" * 30)

    else:
        print(f"Results for {model_name}:")
        print(f"Mean Test Score %: {100*((10**(np.sqrt(result['best_score'])))-1)}")
        print("=" * 30)

Looks like our best model so far is the Elastic Net Regressor. Maybe we can discover more about the data by looking at the feature importances of the model.

## Getting Feature importances of the best model so far

In [ ]:
# Get feature importances
best_model = sorted_results[1][1]['best_model']

# Get best model scores
print(f"Best model scores: {sorted_results[1][1]['best_score']}")
best_model_scores = sorted_results[1][1]['best_score']

# Get feature importances from best model
importances = best_model.feature_importances_

# Get feature names from Xtrain
feature_names = model_data.drop(columns=['SalePrice']).copy().columns

# Sort features by importance in descending order
indices = importances.argsort()[::-1]

# Create a bar chart
plt.figure(figsize=(40, 15))
plt.title("Feature Importances", fontsize=24)  # Increase title fontsize
bars = plt.bar(range(Xtrain.shape[1]), importances[indices], align="center")

# Change color of bars with absolute values greater than 0.6
for i, bar in enumerate(bars):
    if abs(importances[indices][i]) > 0.05:
        bar.set_color('green')

plt.xticks(range(Xtrain.shape[1]), feature_names[indices], rotation=90, fontsize=16)  # Increase x-axis fontsize
plt.yticks(fontsize=16)  # Increase y-axis fontsize
plt.xlim([-1, Xtrain.shape[1]])
plt.tight_layout()
plt.show()



## What about combining models?

Ensemble methods combine the predictions of multiple machine learning models to improve overall predictive performance. In the case of StackingRegressor, it combines the outputs of multiple regression models to make more accurate predictions. We can combine both best models on the grid search above to create a stacked model.

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('ElasticNet', ElasticNet(**params['ElasticNet'])),
    ('DecisionTree', DecisionTreeRegressor(**params['DecisionTreeRegressor'])),
]

# Define the StackingRegressor
StackingRegressorModel = StackingRegressor(estimators=estimators,
                                           final_estimator=RandomForestRegressor(**params['RandomForestRegressor']))

# Train the model on training data
StackingRegressorModel.fit(X=Xtrain, y=ytrain)
StackingRegressorScore = -cross_val_score(StackingRegressorModel, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error')

# Get the mean error
meanError = 100*((10**(np.sqrt(np.mean(StackingRegressorScore))))-1)
print("StackingRegressor Results: ")
print(f"Mean Test Score %: {meanError}")

Woow! Looks like our model really improved by combining different models. Lets see if we can statistically prove that the difference is significant.

## Hypothesis testing: is the model really better?

In [ ]:
# Cross val training Random Forest Model best model to get list of scores
best_model.fit(X=Xtrain, y=ytrain)
RandomForestScore = -cross_val_score(best_model, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error')

# Get the mean error
meanError = 100*((10**(np.sqrt(np.mean(RandomForestScore))))-1)
print("RandomForst Results: ")
print(f"Mean Test Score %: {meanError}")

In [ ]:
from scipy import stats

# Performance scores for StackingRegressor and Elastic Net
stacking_scores = StackingRegressorScore  # Replace with actual scores
random_forest_scores = RandomForestScore  # Replace with actual scores

# Calculate performance differences
differences = [stacking - elastic_net for stacking, elastic_net in zip(stacking_scores, elastic_net_scores)]

# Perform a paired t-test
t_statistic, p_value = stats.ttest_rel(stacking_scores, random_forest_scores)

alpha = 0.05  # Set your significance level

# Interpret the results
if p_value < alpha:
    print("There is a significant difference in performance between Stacking and Random Forest.")
else:
    print("There is no significant difference in performance between the models.")

## WE DID IT!

By combining models we created a new one that is significantly better than the previous ones.

# Testing the model on the test set

In [497]:
from sklearn.metrics import mean_squared_error

# Fit and transform the stacked model                             
StackingRegressorModel.fit_transform(X=Xtrain, y=ytrain)

# Predict target values
y_pred = StackingRegressorModel.predict(Xtest)

# Get the mean error
mean_error = mean_squared_error(ytest, y_pred)

# Print mean error
mean_relative_error = 100*((10**(np.sqrt(mean_error)))-1)
print(f'Mean Absolute Error of StackingRegressor: {round(mean_relative_error, 2)}%')

Mean Absolute Error of StackingRegressor: 12.77%


# Training the model on the whole dataset and saving it

In [508]:
import joblib
StackingRegressorModel.fit_transform(X=X, y=y)

# Save the model and create the file if not exists
model_path = pathlib.Path.cwd().parent / 'models' / 'stacking_model.pkl'
model_path.parent.mkdir(parents=True, exist_ok=True)
joblib.dump(model, model_path)

['/home/victor/insper/6-sem/ml/ames_AndreVictor/models/stacking_model.pkl']